## **4. PyTorch训练相关**

### **4.1 损失函数与反向传播**

损失函数（Loss Function）是机器学习中的一个核心概念，它用于衡量模型预测的输出与实际目标值之间的差异。损失函数计算预测值与真实值之间的误差，这个误差值反映了模型当前的性能：误差越小，表明模型的预测结果越准确。

此外，损失函数不仅提供了一种量化模型性能的方法，还是模型训练过程中参数优化的依据。在训练过程中，通过不断计算损失函数，我们可以使用如梯度下降等优化算法调整模型参数。这里的关键技术是反向传播（Backpropagation），它允许误差从输出层向输入层逐层传递，逐渐调整各层的权重和偏置，从而最小化整体的损失函数。通过这个过程，模型逐渐学习并改善，最终能够更准确地预测或分类。

**$L1损失函数$**

L1 损失函数，也称为最小绝对误差（Least Absolute Deviations, LAD）或 L1 范数损失，是用于回归模型的一种损失函数。它通过计算预测值（$\hat{y}_i$）与真实值（$y_i$）之间差的绝对值的总和来度量模型的预测误差。

L1 损失函数的数学表达式如下：
$$
L = \sum_{i=1}^n |\hat{y}_i - y_i|
$$
其中，$n$ 是数据点的数量。

L1 损失函数的主要特点是对异常值（outliers）不那么敏感，因为它不像平方误差那样对较大的误差赋予过高的惩罚。这使得使用 L1 损失的模型在面对异常值时更为稳健。同时，L1 损失可能导致回归模型的解在统计上是非常稳健的，但它可能存在多个解，因为它的优化是一个非平滑问题。

假设我们有以下两个向量：
- 预测值向量：$ \hat{y} = [100, 200, 300, 400, 500] $
- 实际值向量：$ y = [90, 210, 310, 380, 480] $

$$
\begin{align*}
L1 \; Loss &= |\hat{y}_1 - y_1| + |\hat{y}_2 - y_2| + |\hat{y}_3 - y_3| + |\hat{y}_4 - y_4| + |\hat{y}_5 - y_5|\\
&= |100 - 90| + |200 - 210| + |300 - 310| + |400 - 380| + |500 - 480| \\
&= 70
\end{align*}
$$

In [1]:
import torch
from torch.nn import L1Loss

inputs = torch.tensor([1,2,3], dtype=torch.float32)
targets = torch.tensor([1,2,5], dtype=torch.float32)

# 第一个数字1表示batch_size，即这个批次中只包含一个样本
# 第二个数字1表示通道数，即表示只有一个单通道
# 第三个数字1表示数据的高度，即高度为1，常用于非图像数据或将数据视为一维数组时
# 第四个数字3表示数据的宽度，可以视为向量的长度
inputs = torch.reshape(inputs, (1,1,1,3))
targets = torch.reshape(targets, (1,1,1,3))

loss = L1Loss()  # 默认计算均值
result = loss(inputs,targets)
result

tensor(0.6667)

具体的元素损失为：
- $|\hat{y}_1 - y_1| = |1 - 1| = 0$
- $|\hat{y}_2 - y_2| = |2 - 2| = 0$
- $|\hat{y}_3 - y_3| = |3 - 5| = 2$

如果计算总和：
$$
\text{Sum L1 Loss} = 0 + 0 + 2 = 2
$$

**但`L1Loss`默认为求均值（mean）**，所以实际计算为：
$$
\text{Mean L1 Loss} = \frac{0 + 0 + 2}{3} = \frac{2}{3}
$$

In [2]:
loss = L1Loss(reduction='sum') # 修改为计算总和
result = loss(inputs,targets)
print(result)

tensor(2.)


**$MSE损失函数$**

MSE（Mean Squared Error，均方误差）损失函数是一种常用的回归模型损失函数。它用于评估模型预测值与实际值之间的差异，其计算公式如下：

$$
\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$

其中：
- $ n $ 是样本的数量。
- $ y_i $ 是第 $ i $ 个样本的实际值。
- $ \hat{y}_i $ 是第 $ i $ 个样本的预测值。

MSE的值越小，表示模型的预测结果与实际结果越接近。MSE损失函数具有以下几个特点：

1. **平滑性**：MSE损失函数对小偏差较为敏感，因此在优化过程中能够使得预测值逐渐接近实际值。
2. **凸性**：MSE是一个凸函数，这使得许多优化算法（如梯度下降）能够更容易找到全局最优解。
3. **惩罚大误差**：由于平方的存在，MSE对大误差有更高的惩罚，这有助于减少较大的预测误差。

MSE在机器学习中非常常用，尤其在线性回归、神经网络等回归问题中，用来衡量模型的性能。

假设我们有以下两个向量：
- 预测值向量：$ \hat{y} = [105, 205, 295, 405, 495] $
- 实际值向量：$ y = [100, 200, 300, 400, 500] $

$$
\begin{align*}
MSE \; Loss &= \frac{1}{n} \sum_{i=1}^n (\hat{y}_i - y_i)^2 \\
&= \frac{1}{5} \left[ (\hat{y}_1 - y_1)^2 + (\hat{y}_2 - y_2)^2 + (\hat{y}_3 - y_3)^2 + (\hat{y}_4 - y_4)^2 + (\hat{y}_5 - y_5)^2 \right] \\
&= \frac{1}{5} \left[ (105 - 100)^2 + (205 - 200)^2 + (295 - 300)^2 + (405 - 400)^2 + (495 - 500)^2 \right] \\
&= 25
\end{align*}

In [3]:
import torch
from torch import nn

inputs = torch.tensor([1,2,3],dtype=torch.float32)
targets = torch.tensor([1,2,5],dtype=torch.float32)

inputs = torch.reshape(inputs,(1,1,1,3))
targets = torch.reshape(targets,(1,1,1,3))

loss_mse = nn.MSELoss()
result_mse = loss_mse(inputs,targets)
print(result_mse)

tensor(1.3333)


**$交叉熵损失函数$**

交叉熵损失函数（Cross-Entropy Loss），也称作对数损失（Log Loss），是分类问题中常用的损失函数。它用于衡量预测概率分布与实际标签分布之间的差异。交叉熵损失函数的公式如下：

对于二分类问题，交叉熵损失函数定义为：

$$
L(y, \hat{y}) = - \frac{1}{n} \sum_{i=1}^{n} \left[ y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i) \right]
$$

其中：
- $ n $ 是样本的数量。
- $ y_i $ 是第 $ i $ 个样本的实际标签，取值为0或1。
- $ \hat{y}_i $ 是第 $ i $ 个样本的预测概率，表示预测为类别1的概率。

对于多分类问题，交叉熵损失函数定义为：

$$ L(y, \hat{y}) = - \frac{1}{n} \sum_{i=1}^{n} \sum_{c=1}^{C} y_{i,c} \log(\hat{y}_{i,c}) $$

其中：
- $ C $ 是类别的数量。
- $ y_{i,c} $ 是第 $ i $ 个样本的实际标签，使用one-hot编码，如果样本 $ i $ 属于类别 $ c $，则 $ y_{i,c} = 1 $，否则 $ y_{i,c} = 0 $。
- $ \hat{y}_{i,c} $ 是第 $ i $ 个样本预测为类别 $ c $ 的概率。

交叉熵损失函数具有以下几个特点：

1. **衡量不确定性**：交叉熵损失能够很好地衡量预测的概率分布与实际分布之间的差异，特别适用于概率预测。
2. **凸性**：交叉熵损失函数是一个凸函数，这使得许多优化算法（如梯度下降）能够有效地找到全局最优解。
3. **关注分类准确性**：交叉熵损失函数对错误分类有较高的惩罚，使得模型在训练过程中更加注重提高分类准确性。

交叉熵损失函数在深度学习和机器学习的分类任务中非常常用，尤其是在神经网络中的分类问题，如图像分类、文本分类等。

假设有一个三分类问题，模型需要对三种不同的类别进行分类。我们有以下预测概率向量和实际类标签向量：

- **预测概率向量**（模型输出的概率，已通过softmax函数转换）：$ \hat{y} = [0.2, 0.7, 0.1] $
- **实际类标签向量**（使用one-hot编码）：$ y = [0, 1, 0] $

在这个例子中，模型预测第二个类别的概率最高（0.7），而实际的类别也是第二个。

**交叉熵损失计算**：

$$
CE \; Loss = -\sum_{i=1}^n y_i \log(\hat{y}_i)
$$
其中，$ n $ 是类别的数量。

**应用于给定数据**：
$$
\begin{align*}
CE \; Loss &= - [y_1 \log(\hat{y}_1) + y_2 \log(\hat{y}_2) + y_3 \log(\hat{y}_3)] \\
&= - [0 \log(0.2) + 1 \log(0.7) + 0 \log(0.1)] \\
&= - [0 + \log(0.7) + 0] \\
&= -\log(0.7) \\
&≈ 0.3567
\end{align*}
$$

In [4]:
import torch
from torch import nn

x = torch.tensor([0.1,0.2,0.3])
y = torch.tensor([1])

x = torch.reshape(x,(1,3)) # 1表示batch_size，3表示有3类

loss_cross = nn.CrossEntropyLoss()
result_cross = loss_cross(x,y)
print(result_cross)

tensor(1.1019)


在 PyTorch 中使用 `nn.CrossEntropyLoss` 时，这个损失函数计算涉及两个主要步骤：softmax 应用于预测值，然后计算交叉熵损失。给定的例子中，`x` 是模型输出的原始得分（通常称为 logits），而 `y` 是目标类别的索引。

**Softmax 归一化**：

先对 logits `x` 应用 softmax 函数来转换成概率值。Softmax 函数定义为：
$$
\text{softmax}(z_i) = \frac{e^{z_i}}{\sum_{j} e^{z_j}}
$$
其中，$z_i$ 是向量中的第 $i$ 个元素。

对于输入 `x = [0.1, 0.2, 0.3]`，softmax 应用于每个元素如下：
- $e^{0.1} \approx 1.105$
- $e^{0.2} \approx 1.221$
- $e^{0.3} \approx 1.349$

和为 $1.105 + 1.221 + 1.349 \approx 3.675$

所以，归一化后的概率为：
- $\text{softmax}(0.1) \approx \frac{1.105}{3.675} \approx 0.301$
- $\text{softmax}(0.2) \approx \frac{1.221}{3.675} \approx 0.332$
- $\text{softmax}(0.3) \approx \frac{1.349}{3.675} \approx 0.367$

**交叉熵损失计算**：

接下来，根据实际的类别标签 `y = [1]`，计算交叉熵损失。交叉熵损失的计算公式是：
$$
\text{cross entropy loss} = -\log(\text{predicted probability of the true class})
$$

对于 `y = 1`，真实类别的预测概率是 $0.332$ (对应索引1的概率)。因此：
$$
\text{cross entropy loss} = -\log(0.332)
$$

计算 $\log(0.332)$，假设以自然对数为底：
$$
\log(0.332) \approx -1.101
$$

所以，损失为：
$$
\text{cross entropy loss} \approx 1.101
$$

In [5]:
# 上一小节中建立的神经网络
import torch
import torchvision
from torch import nn 
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, Sequential
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

dataset = torchvision.datasets.CIFAR10("./data", train=False,
                                       transform=torchvision.transforms.ToTensor(),
                                       download=False)       
dataloader = DataLoader(dataset, batch_size=4, drop_last=True)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()        
        self.model1 = Sequential(
            Conv2d(3,32,5,padding=2),
            MaxPool2d(2),
            Conv2d(32,32,5,padding=2),
            MaxPool2d(2),
            Conv2d(32,64,5,padding=2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024,64),
            Linear(64,10)
        )
        
    def forward(self, x):
        x = self.model1(x)
        return x

2025-03-28 15:56:50.046705: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-28 15:56:50.080820: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743148610.102106   18813 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743148610.108571   18813 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743148610.125811   18813 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [7]:
network = NeuralNetwork()
for data in dataloader:
    imgs, targets = data
    outputs = network(imgs)
    print(outputs)
    print(targets)

tensor([[-0.0755, -0.0344, -0.0421, -0.0639,  0.0781, -0.0721,  0.0033, -0.0480,
          0.0197, -0.0538],
        [-0.1005, -0.0458, -0.0327, -0.0632,  0.0472, -0.0672, -0.0173, -0.0444,
          0.0212, -0.0389],
        [-0.0852, -0.0511, -0.0298, -0.0679,  0.0600, -0.0686, -0.0011, -0.0394,
          0.0037, -0.0517],
        [-0.0938, -0.0429, -0.0344, -0.0741,  0.0612, -0.0703, -0.0030, -0.0544,
          0.0060, -0.0522]], grad_fn=<AddmmBackward0>)
tensor([3, 8, 8, 0])
tensor([[-7.9836e-02, -3.6896e-02, -4.4793e-02, -5.9564e-02,  8.3184e-02,
         -7.6289e-02,  4.8858e-05, -3.5773e-02,  1.0583e-02, -4.8784e-02],
        [-9.4209e-02, -2.3460e-02, -4.2744e-02, -5.9221e-02,  8.0791e-02,
         -7.2566e-02, -7.3863e-03, -5.1127e-02,  2.8264e-02, -5.2870e-02],
        [-7.3297e-02, -1.6324e-02, -4.2130e-02, -6.3667e-02,  8.0632e-02,
         -6.2537e-02,  2.1689e-02, -4.4509e-02,  1.0354e-02, -8.0292e-02],
        [-8.5177e-02, -3.0663e-02, -3.9631e-02, -6.4698e-02,  8.4037e

In [8]:
# 计算损失函数
loss = nn.CrossEntropyLoss() # 交叉熵    
network = NeuralNetwork()
for data in dataloader:
    imgs, targets = data
    outputs = network(imgs)
    result_loss = loss(outputs, targets) # 计算实际输出与目标输出的差距
    print(result_loss)

tensor(2.3343, grad_fn=<NllLossBackward0>)
tensor(2.1982, grad_fn=<NllLossBackward0>)
tensor(2.2704, grad_fn=<NllLossBackward0>)
tensor(2.3505, grad_fn=<NllLossBackward0>)
tensor(2.3291, grad_fn=<NllLossBackward0>)
tensor(2.2693, grad_fn=<NllLossBackward0>)
tensor(2.3118, grad_fn=<NllLossBackward0>)
tensor(2.2620, grad_fn=<NllLossBackward0>)
tensor(2.3226, grad_fn=<NllLossBackward0>)
tensor(2.3042, grad_fn=<NllLossBackward0>)
tensor(2.2653, grad_fn=<NllLossBackward0>)
tensor(2.2790, grad_fn=<NllLossBackward0>)
tensor(2.3025, grad_fn=<NllLossBackward0>)
tensor(2.3481, grad_fn=<NllLossBackward0>)
tensor(2.3025, grad_fn=<NllLossBackward0>)
tensor(2.3173, grad_fn=<NllLossBackward0>)
tensor(2.2864, grad_fn=<NllLossBackward0>)
tensor(2.3126, grad_fn=<NllLossBackward0>)
tensor(2.3366, grad_fn=<NllLossBackward0>)
tensor(2.3453, grad_fn=<NllLossBackward0>)
tensor(2.3070, grad_fn=<NllLossBackward0>)
tensor(2.3504, grad_fn=<NllLossBackward0>)
tensor(2.3081, grad_fn=<NllLossBackward0>)
tensor(2.25

反向传播是一种在神经网络训练中使用的核心算法，其目的是通过计算损失函数关于网络参数的梯度来优化这些参数。通过这种方式，反向传播能够有效地调整网络中的权重和偏置，以最小化在训练过程中计算得到的损失。这个过程从网络的输出层开始，逐层向输入层传递，更新每一层的参数以减少总体损失。通过反复迭代这个过程，神经网络逐渐学习并改善其性能，以更准确地模拟或预测数据。

In [8]:
# 反向传播
loss = nn.CrossEntropyLoss() # 交叉熵
network = NeuralNetwork()
for data in dataloader:
    imgs, targets = data
    outputs = network(imgs)
    # 计算预测输出与实际标签之间的损失
    result_loss = loss(outputs, targets)
    # 计算出来的loss值有backward方法属性
    # 执行反向传播，计算损失对各参数的梯度
    result_loss.backward()

在神经网络的训练过程中，**反向传播**是一个关键的步骤，它负责计算损失函数相对于网络参数（如权重和偏置）的梯度。这些梯度用于更新网络参数，以最小化损失函数。这个过程主要包括以下几个方面：

1. **计算梯度**：
- 调用 `.backward()` 方法时，PyTorch 会自动计算当前损失值关于网络参数的梯度。这意味着，它会计算出损失函数如何随着每个参数的变化而变化，这些变化值（梯度）指示了如何调整参数以减少损失。

2. **梯度属性（`grad`）**：
- 在PyTorch中，每个张量（Tensor）对象都有一个 `.grad` 属性，这个属性用来存储关于这个张量的梯度。对于网络中的参数（通常是张量），在执行 `.backward()` 之前，这些参数的 `.grad` 属性是空的，因为梯度还没有被计算出来。
- 当执行了 `.backward()` 后，这些参数的 `.grad` 属性将包含对应的梯度值。这些梯度表示了在参数空间中，损失函数增加或减少的最快方向。

3. **使用梯度更新参数**：
- 一旦梯度被计算出来并存储在参数的 `.grad` 属性中，优化器（如 SGD, Adam 等）可以使用这些梯度来更新网络的参数。优化器会根据梯度方向和设定的学习率等参数，调整网络权重，目的是朝着减少损失的方向优化。

### **4.2 优化器**

在训练神经网络时，使用损失函数来衡量模型的预测结果与实际值之间的差异。为了改善模型的预测准确性，我们需要根据这个损失来调整模型的参数（如权重和偏置）。这里的过程涉及到几个关键步骤：

1. **反向传播与梯度计算**
当通过损失函数计算出模型的误差后，可以通过调用损失值的 `.backward()` 方法来自动计算所有相关参数的梯度。这个过程叫做**反向传播**。在反向传播过程中，计算从损失函数开始，逆向通过网络，根据链式法则计算出每个参数对损失的影响程度，即参数的梯度。梯度描述了损失相对于各参数的变化率，告诉我们如何改变参数以减小损失。

2. **使用优化器调整参数**
一旦梯度被计算出来，就可以使用优化器（如SGD、Adam等）根据这些梯度来更新模型的参数。优化器根据梯度的方向和大小来调整参数，以期减少总损失。例如，如果一个参数的梯度是正的，减小该参数的值可能会减少损失；如果梯度是负的，增加该参数的值可能会有帮助。

3. **梯度清零**
在PyTorch中，每次调用 `.backward()` 方法时，梯度是累加到已存在的梯度上的。这意味着如果不手动清零梯度，每次梯度计算的结果会与前一次的梯度相加，导致不正确的更新方向和大小。因此，在每次训练迭代开始前，需要清空（归零）梯度缓存，以确保每次参数更新只基于最新计算的梯度。这通常通过调用优化器的 `zero_grad()` 方法实现。

In [9]:
# 之前创建的神经网络
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, Sequential

dataset = torchvision.datasets.CIFAR10("./data", train=False,
                                       transform=torchvision.transforms.ToTensor(),
                                       download=False)
dataloader = DataLoader(dataset, batch_size=64, drop_last=True)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()        
        self.model1 = Sequential(
            Conv2d(3, 32, 5, padding=2),
            MaxPool2d(2),
            Conv2d(32, 32, 5, padding=2),
            MaxPool2d(2),
            Conv2d(32, 64, 5, padding=2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024, 64),
            Linear(64, 10)
        )
        
    def forward(self, x):
        x = self.model1(x)
        return x

In [13]:
# 神经网络优化一轮
loss = nn.CrossEntropyLoss() # 交叉熵    
network = NeuralNetwork()
optim = torch.optim.SGD(network.parameters(), lr=0.01)      # 随机梯度下降优化器
for data in dataloader:
    imgs, targets = data
    outputs = network(imgs)
    result_loss = loss(outputs, targets)    # 计算实际输出与目标输出的差距
    optim.zero_grad()   # 梯度清零
    result_loss.backward()  # 反向传播，计算损失函数的梯度
    optim.step()    # 根据梯度，对网络的参数进行调优
    print(result_loss)  # 对数据只看了一遍，只看了一轮，所以loss下降不大

tensor(2.3104, grad_fn=<NllLossBackward0>)
tensor(2.3069, grad_fn=<NllLossBackward0>)
tensor(2.2897, grad_fn=<NllLossBackward0>)
tensor(2.2988, grad_fn=<NllLossBackward0>)
tensor(2.3106, grad_fn=<NllLossBackward0>)
tensor(2.3026, grad_fn=<NllLossBackward0>)
tensor(2.2907, grad_fn=<NllLossBackward0>)
tensor(2.3039, grad_fn=<NllLossBackward0>)
tensor(2.2956, grad_fn=<NllLossBackward0>)
tensor(2.3042, grad_fn=<NllLossBackward0>)
tensor(2.3081, grad_fn=<NllLossBackward0>)
tensor(2.3069, grad_fn=<NllLossBackward0>)
tensor(2.3117, grad_fn=<NllLossBackward0>)
tensor(2.3090, grad_fn=<NllLossBackward0>)
tensor(2.2999, grad_fn=<NllLossBackward0>)
tensor(2.3216, grad_fn=<NllLossBackward0>)
tensor(2.3044, grad_fn=<NllLossBackward0>)
tensor(2.2990, grad_fn=<NllLossBackward0>)
tensor(2.3164, grad_fn=<NllLossBackward0>)
tensor(2.3012, grad_fn=<NllLossBackward0>)
tensor(2.3071, grad_fn=<NllLossBackward0>)
tensor(2.3029, grad_fn=<NllLossBackward0>)
tensor(2.3213, grad_fn=<NllLossBackward0>)
tensor(2.30

In [14]:
# 神经网络优化多轮
loss = nn.CrossEntropyLoss() # 交叉熵    
network = NeuralNetwork()
optim = torch.optim.SGD(network.parameters(), lr=0.01)   # 随机梯度下降优化器
for epoch in range(20):
    running_loss = 0.0
    for data in dataloader:
        imgs, targets = data
        outputs = network(imgs)
        result_loss = loss(outputs, targets)    # 计算实际输出与目标输出的差距
        optim.zero_grad()   # 梯度清零
        result_loss.backward()  # 反向传播，计算损失函数的梯度
        optim.step()    # 根据梯度，对网络的参数进行调优
        # print(result_loss)
        running_loss = running_loss + result_loss
    print(running_loss) # 对这一轮所有误差的总和

tensor(358.4160, grad_fn=<AddBackward0>)
tensor(355.2607, grad_fn=<AddBackward0>)
tensor(342.9598, grad_fn=<AddBackward0>)
tensor(321.7895, grad_fn=<AddBackward0>)
tensor(309.7653, grad_fn=<AddBackward0>)
tensor(299.2832, grad_fn=<AddBackward0>)
tensor(289.7178, grad_fn=<AddBackward0>)
tensor(282.1351, grad_fn=<AddBackward0>)
tensor(275.2144, grad_fn=<AddBackward0>)
tensor(268.8350, grad_fn=<AddBackward0>)
tensor(262.8195, grad_fn=<AddBackward0>)
tensor(257.1351, grad_fn=<AddBackward0>)
tensor(251.8302, grad_fn=<AddBackward0>)
tensor(246.9349, grad_fn=<AddBackward0>)
tensor(242.4683, grad_fn=<AddBackward0>)
tensor(238.3860, grad_fn=<AddBackward0>)
tensor(234.6182, grad_fn=<AddBackward0>)
tensor(231.1008, grad_fn=<AddBackward0>)
tensor(227.7249, grad_fn=<AddBackward0>)
tensor(224.4496, grad_fn=<AddBackward0>)


In [16]:
# 学习率优化
loss = nn.CrossEntropyLoss() # 交叉熵    
network = NeuralNetwork()
optim = torch.optim.SGD(network.parameters(),lr=0.01)   # 随机梯度下降优化器
# 每过 step_size 更新一次优化器，更新是学习率为原来的学习率的的 1.0 倍    
scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=5, gamma=1.0)
for epoch in range(20):
    running_loss = 0.0
    for data in dataloader:
        imgs, targets = data
        outputs = network(imgs)
        result_loss = loss(outputs, targets)    # 计算实际输出与目标输出的差距
        optim.zero_grad()   # 梯度清零
        result_loss.backward()  # 反向传播，计算损失函数的梯度
        optim.step()    # 根据梯度，对网络的参数进行调优
        scheduler.step()    # 学习率太小了，所以20个轮次后，相当于没走多少
        running_loss = running_loss + result_loss
    print(running_loss) # 对这一轮所有误差的总和

tensor(358.6636, grad_fn=<AddBackward0>)
tensor(354.9099, grad_fn=<AddBackward0>)
tensor(336.0622, grad_fn=<AddBackward0>)
tensor(319.5813, grad_fn=<AddBackward0>)
tensor(310.9790, grad_fn=<AddBackward0>)
tensor(302.5688, grad_fn=<AddBackward0>)
tensor(292.5781, grad_fn=<AddBackward0>)
tensor(284.5847, grad_fn=<AddBackward0>)
tensor(277.1803, grad_fn=<AddBackward0>)
tensor(270.2680, grad_fn=<AddBackward0>)
tensor(264.0457, grad_fn=<AddBackward0>)
tensor(258.5112, grad_fn=<AddBackward0>)
tensor(253.6092, grad_fn=<AddBackward0>)
tensor(249.2558, grad_fn=<AddBackward0>)
tensor(245.3011, grad_fn=<AddBackward0>)
tensor(241.5980, grad_fn=<AddBackward0>)
tensor(238.0139, grad_fn=<AddBackward0>)
tensor(234.5474, grad_fn=<AddBackward0>)
tensor(231.1807, grad_fn=<AddBackward0>)
tensor(227.8823, grad_fn=<AddBackward0>)


### **4.3 网络模型的使用及修改**

In [22]:
dir(torchvision.models)

['AlexNet',
 'AlexNet_Weights',
 'ConvNeXt',
 'ConvNeXt_Base_Weights',
 'ConvNeXt_Large_Weights',
 'ConvNeXt_Small_Weights',
 'ConvNeXt_Tiny_Weights',
 'DenseNet',
 'DenseNet121_Weights',
 'DenseNet161_Weights',
 'DenseNet169_Weights',
 'DenseNet201_Weights',
 'EfficientNet',
 'EfficientNet_B0_Weights',
 'EfficientNet_B1_Weights',
 'EfficientNet_B2_Weights',
 'EfficientNet_B3_Weights',
 'EfficientNet_B4_Weights',
 'EfficientNet_B5_Weights',
 'EfficientNet_B6_Weights',
 'EfficientNet_B7_Weights',
 'EfficientNet_V2_L_Weights',
 'EfficientNet_V2_M_Weights',
 'EfficientNet_V2_S_Weights',
 'GoogLeNet',
 'GoogLeNetOutputs',
 'GoogLeNet_Weights',
 'Inception3',
 'InceptionOutputs',
 'Inception_V3_Weights',
 'MNASNet',
 'MNASNet0_5_Weights',
 'MNASNet0_75_Weights',
 'MNASNet1_0_Weights',
 'MNASNet1_3_Weights',
 'MaxVit',
 'MaxVit_T_Weights',
 'MobileNetV2',
 'MobileNetV3',
 'MobileNet_V2_Weights',
 'MobileNet_V3_Large_Weights',
 'MobileNet_V3_Small_Weights',
 'RegNet',
 'RegNet_X_16GF_Weights'

In [26]:
import torchvision
from torch import nn

# # ImageNet目前不支持公开访问
# train_data = torchvision.datasets.ImageNet('./data', split='train', download=True, 
#                                            transform=torchvision.transforms.ToTensor())

# 下载并加载预训练的VGG16模型，这些参数是在ImageNet数据集上训练得到的
vgg16_true = torchvision.models.vgg16(pretrained=True)
# 下载VGG16模型但不加载预训练权重
vgg16_false = torchvision.models.vgg16(pretrained=False)

# 打印预训练的VGG16模型结构，查看各层参数和设置
vgg16_true

c:\Users\Daniel Johnson\anaconda3\envs\PyTorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

VGG16 模型默认的输出特征数量为 1000，意味着它能够区分 1000 个不同的类别。我们现在对这一设置进行调整，将其修改为进行十分类，即令输出特征数量为 10。

In [17]:
# 添加一层，in_features为1000，out_features为10
train_data = torchvision.datasets.CIFAR10("./data", train=True,
                                          transform=torchvision.transforms.ToTensor(),
                                          download=False)    

vgg16_true = torchvision.models.vgg16(pretrained=True)
# 在VGG16后面添加一个线性层（添加了一个add_linear的module）
vgg16_true.add_module('add_linear',nn.Linear(1000,10))

vgg16_true

/home/cpl27272/ENTER/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/cpl27272/ENTER/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/cpl27272/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:17<00:00, 32.1MB/s] 


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [18]:
# 在vgg16模型的classifier这个module中添加一个额外的线性层
vgg16_true = torchvision.models.vgg16(pretrained=True)
vgg16_true.classifier.add_module('add_linear', nn.Linear(1000, 10))
vgg16_true

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [19]:
vgg16_false = torchvision.models.vgg16(pretrained=False)
vgg16_false

/home/cpl27272/ENTER/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [20]:
# 直接修改vgg16模型的classifier这个module中的线性层
vgg16_false.classifier[6] = nn.Linear(4096, 10)
vgg16_false

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### **4.4 网络模型的保存与读取**

In [22]:
import torch
import torchvision
from torch import nn

vgg16 = torchvision.models.vgg16(pretrained=False)
# 模型保存方式 1
# 保存了模型结构 + 模型参数
torch.save(vgg16,"./model/vgg16_method1.pth")
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [23]:
# 与保存方式1对应的加载方式
model1 = torch.load('./model/vgg16_method1.pth')
model1

/tmp/ipykernel_18813/1088995122.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model1 = torch.load('./model/vgg16_method1.pth')


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [25]:
# 模型保存方式 2（官方推荐）
# 将网络模型的参数保存为字典，不再保存网络模型结构
torch.save(vgg16.state_dict(), "./model/vgg16_method2.pth")

In [26]:
# 模型加载
# 导入的是模型参数，没有模型结构的信息
model2 = torch.load("./model/vgg16_method2.pth")
model2

/tmp/ipykernel_18813/1914910030.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model2 = torch.load("./model/vgg16_method2.pth")


OrderedDict([('features.0.weight',
              tensor([[[[-9.1610e-03, -1.6100e-02, -1.2155e-01],
                        [ 3.4295e-02, -7.5555e-02,  8.1976e-02],
                        [ 5.1360e-02, -4.0354e-02, -8.9726e-02]],
              
                       [[-1.1079e-01,  1.1473e-01,  1.2020e-01],
                        [-1.2059e-02, -6.2140e-03,  1.0462e-01],
                        [ 2.6397e-05, -2.5063e-02,  1.7382e-02]],
              
                       [[ 5.3922e-02, -5.0677e-02,  6.6099e-02],
                        [ 7.1893e-02,  4.3690e-02, -1.0109e-01],
                        [-5.4995e-03, -3.3707e-02,  7.2347e-02]]],
              
              
                      [[[ 3.9737e-02, -2.0369e-02, -6.7276e-02],
                        [ 3.1809e-02,  4.7963e-02,  1.1368e-02],
                        [-5.4050e-04, -2.0309e-02, -7.4675e-02]],
              
                       [[-4.7939e-02,  1.1358e-01, -1.4235e-02],
                        [ 3.1063e-02, -8

In [27]:
# 与保存方式2对应的加载方式
# 网络结构，没有参数
vgg16 = torchvision.models.vgg16(pretrained=False)
# 将模型参数导入到模型结构中
vgg16.load_state_dict(torch.load("./model/vgg16_method2.pth"))
print(vgg16)

/tmp/ipykernel_18813/2597988090.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vgg16.load_state_dict(torch.load("./model/vgg16_method2.pth"))


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [6]:
# # 创建一个网络结构
# class NeuralNetwork(nn.Module):
#     def __init__(self):
#         super(NeuralNetwork,self).__init__()
#         self.conv1 = nn.Conv2d(3, 64, kernel_size=3)
        
#     def forward(self,x):
#         x = self.conv1(x)
#         return x

# network = NeuralNetwork()
# # 使用方式1保存网络模型
# torch.save(network, "./model/network_method1.pth")

网络陷阱——失败加载模型

Restart kernel，再运行下面的代码，即下面为第1个代码块运行，无法直接导入网络模型。。

In [7]:
# 陷阱1
# 无法直接加载用方式1保存的网络结构
# 代码需要定义网络结果，即上一个代码块取消注释即可
model = torch.load("./model/network_method1.pth") 
model

C:\Users\Daniel Johnson\AppData\Local\Temp\ipykernel_17120\1185946972.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("./model/network_method1.pth")  

AttributeError: Can't get attribute 'NeuralNetwork' on <module '__main__'>

In [8]:
# 需要在加载前还写明网络模型，确保网络模型是我们想要的网络模型
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.conv1 = nn.Conv2d(3,64,kernel_size=3)
        
    def forward(self,x):
        x = self.conv1(x)
        return x
    
# network = NeuralNetwork # 不需要写这一步，不需要创建网络模型    
model = torch.load("./model/network_method1.pth")
model

NeuralNetwork(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
)


C:\Users\Daniel Johnson\AppData\Local\Temp\ipykernel_17120\2381926589.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("./model/network_method1.pth")
